In [102]:
from pyvirtualdisplay import Display
import copy
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

import numpy as np
np.set_printoptions(precision=4, suppress=True)


from diffuser.models.temporal import TemporalUnet
from diffuser.utils.arrays import report_parameters
from diffuser.datasets.sequence import SequenceDataset
import torch.utils.data as tdata
import pytorch_lightning as pl
import gym
import matplotlib.pyplot as plt
import PIL
from PIL import Image, ImageOps, ImageChops

# from diffuser.datasets import sequence_dataset

import pickle

dataset = pickle.load(open("mc.pkl", "rb"))


# class MountainCarDataModule(pl.LightningDataModule):
#     def __init__(self, train_transforms=None, val_transforms=None, test_transforms=None, dims=None):
#         super().__init__(train_transforms, val_transforms, test_transforms, dims)

#     def prepare_data(self) -> None:

#         return super().prepare_data()
env = gym.make("MountainCarContinuous-v0")
d = SequenceDataset(ds=dataset, env="MountainCarContinuous-v0")
ld = tdata.DataLoader(d, batch_size=3, num_workers=1, shuffle=False, pin_memory=True)

# # model = TemporalUnet(64, 3, 2, 1, dim_mults=(1, 2))
# # print(model)
# # report_parameters(model)


def cycle(dl):
    while True:
        for data in dl:
            yield data


batch = next(cycle(ld))
traj = batch.trajectories[0]

env.unwrapped.state = traj[0, 1:]




<class 'gym.envs.classic_control.continuous_mountain_car.Continuous_MountainCarEnv'>
[ datasets/buffer ] Finalized replay buffer | 610 episodes
[ datasets/buffer ] Fields:
    rewards: (610, 1000, 1)
    terminals: (610, 1000, 1)
    observations: (610, 1000, 2)
    actions: (610, 1000, 1)
    normed_observations: (610, 1000, 2)
    normed_actions: (610, 1000, 1)


In [98]:
dataset

{'rewards': array([-0.0066, -0.    , -0.0113, ..., -0.0091, -0.0088, 99.9999]),
 'terminals': array([False, False, False, ..., False, False,  True]),
 'observations': array([[-0.5371,  0.    ],
        [-0.5374, -0.0003],
        [-0.5376, -0.0002],
        ...,
        [ 0.3899,  0.0216],
        [ 0.411 ,  0.0211],
        [ 0.4317,  0.0207]]),
 'actions': array([[-0.2568],
        [-0.0085],
        [-0.3357],
        ...,
        [ 0.3017],
        [ 0.2966],
        [-0.0307]], dtype=float32)}

In [23]:
traj

tensor([[-0.1973, -0.1965,  0.0253],
        [-0.0804, -0.1968,  0.0212],
        [-0.2344, -0.1971,  0.0225],
        [ 0.1335, -0.1978,  0.0167],
        [-0.0847, -0.1976,  0.0281],
        [ 0.0794, -0.1972,  0.0293],
        [ 0.0311, -0.1962,  0.0381],
        [-0.1262, -0.1946,  0.0446],
        [-0.1284, -0.1931,  0.0436],
        [-0.2555, -0.1917,  0.0423],
        [-0.2873, -0.1909,  0.0350],
        [-0.1058, -0.1908,  0.0261],
        [ 0.0020, -0.1908,  0.0257],
        [-0.0325, -0.1903,  0.0304],
        [-0.0593, -0.1897,  0.0334],
        [-0.1596, -0.1889,  0.0350],
        [-0.0316, -0.1883,  0.0319],
        [-0.1436, -0.1875,  0.0348],
        [-0.2501, -0.1870,  0.0323],
        [-0.1033, -0.1870,  0.0248],
        [-0.2213, -0.1871,  0.0242],
        [-0.1034, -0.1877,  0.0181],
        [-0.1169, -0.1883,  0.0175],
        [ 0.0281, -0.1891,  0.0164],
        [ 0.1275, -0.1893,  0.0221],
        [-0.0005, -0.1887,  0.0324],
        [ 0.0519, -0.1878,  0.0368],
 

In [185]:
STOP = 617

render_env = gym.wrappers.Monitor(env, "test", force=True)
render_env.reset()
render_env.state = render_env.unwrapped.state = render_env.unwrapped.unwrapped.state = copy.deepcopy(Z[0, 1:])
prev_obs = Z[0, 1:]
recorded = []
for _, i in enumerate(Z[:STOP]):
    act = i[:1]
    obs, rew, done, info =render_env.step(act)
    if done:
        break
    recorded.append(np.concatenate([act, prev_obs]))
    prev_obs = obs

render_env.close()
recorded = np.array(recorded)

In [186]:
np.all(Z[:STOP] == recorded)

True

In [179]:
Z[:64] == recorded

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ T

In [181]:

for i in range(len(dataset['terminals'])):
    if dataset['terminals'][i]:
        print(i)
        break

617


In [174]:
Z = np.hstack([dataset['actions'], dataset['observations']])